# Face Detection using OPENCV


The Viola-Jones algorithm is a widely used object detection framework, primarily used for detecting faces in images but applicable to other object recognition tasks as well. It was developed by Paul Viola and Michael Jones in 2001 and has since become a fundamental technique in computer vision.

Here are the key components and steps of the Viola-Jones algorithm:

1. Integral Image: The algorithm starts by computing an integral image representation of the input image. The integral image allows for fast calculation of the sum of pixel values within any rectangular region of the image.

2. Haar-like Features: The core idea of Viola-Jones is to use Haar-like features to represent regions of the image. These features are simple rectangular filters that can be applied at multiple scales and positions within the image. Examples of Haar-like features include edge features, line features, and rectangle features.

3. AdaBoost Training: The algorithm uses a machine learning technique called AdaBoost (Adaptive Boosting) to select a small set of highly discriminative Haar-like features from a large pool of potential features. AdaBoost assigns weights to each training sample and iteratively selects the best features that classify the training data correctly.

4. Cascade Classifier: The selected Haar-like features are organized into a cascade classifier, which consists of multiple stages. Each stage is composed of a subset of the selected features and a threshold. The stages are evaluated sequentially, and if a stage fails to classify an image as non-object, the evaluation stops, saving processing time.

5. Detection and Non-Maximum Suppression: The cascade classifier is applied to the integral image in a sliding window fashion, scanning the entire image at different scales. When a region of the image is classified as a potential object, non-maximum suppression is applied to remove overlapping detections and keep only the most confident one.

6. Post-processing: Once potential object regions are identified, additional post-processing steps may be applied, such as face landmark detection or tracking.


In [ ]:
# Face Recognition
import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') # We load the cascade for the face.
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml') # We load the cascade for the eyes.
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')

def detect(gray, frame): # We create a function that takes as input the image in black and white (gray) and the original image (frame), and that will return the same image with the detector rectangles. 
    faces = face_cascade.detectMultiScale(gray, 1.3, 5) # We apply the detectMultiScale method from the face cascade to locate one or several faces in the image.
    for (x, y, w, h) in faces: # For each detected face:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2) # We paint a rectangle around the face.
        roi_gray = gray[y:y+h, x:x+w] # We get the region of interest in the black and white image.
        roi_color = frame[y:y+h, x:x+w] # We get the region of interest in the colored image.
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 3) # We apply the detectMultiScale method to locate one or several eyes in the image.
        for (ex, ey, ew, eh) in eyes: # For each detected eye:
            cv2.rectangle(roi_color,(ex, ey),(ex+ew, ey+eh), (0, 255, 0), 2) # We paint a rectangle around the eyes, but inside the referential of the face.
        smile = smile_cascade.detectMultiScale(roi_gray,1.7,22)
        for (sx,sy,sw,sh) in smile:
            cv2.rectangle(roi_color,(sx,sy),(sx+sw,sy+sh),(0,0,255),2)
    return frame # We return the image with the detector rectangles.

video_capture = cv2.VideoCapture(0) # We turn the webcam on.

while True: # We repeat infinitely (until break):
    _, frame = video_capture.read() # We get the last frame.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # We do some colour transformations.
    canvas = detect(gray, frame) # We get the output of our detect function.
    cv2.imshow('Video', canvas) # We display the outputs.
    if cv2.waitKey(1) & 0xFF == ord('q'): # If we type on the keyboard:
        break # We stop the loop.
out = cv2.VideoWriter('output.avi', -1, 20.0, (640,480))

while(canvas.isOpened()):
    ret, frame = canvas.read()
    if ret==True:
        frame = cv2.flip(frame,0)

        # write the flipped frame
        out.write(frame)

        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

 # We destroy all the windows inside which the images were displayed.
video_capture.release() # We turn the webcam off.
cv2.destroyAllWindows()